In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
from sklearn.cross_validation import cross_val_score 
from sklearn.linear_model import LogisticRegression 
from sklearn.linear_model import LogisticRegressionCV          
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectFromModel
import operator

/opt/conda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [73]:
finalDf = pd.DataFrame()
test = ['/data/reddit5.csv',
         '/data/reddit6.csv',
         '/data/reddit7.csv']
train=['/data/reddit4.csv',
       '/data/reddit8.csv',
       '/data/reddit9.csv']

import csv
first = True
count = 100000
with open('train.csv', 'w', newline="") as f:
    writer = csv.writer(f)
    for file in train:
        with open(file, 'r', newline="") as f:
            reader = csv.reader(f)
            if first == True:
                writer.writerow(next(reader))
                first = False
            else:
                next(reader)
            for row in reader:
                if count == 0:
                    break
                if(int(row[20]) == 1):
                    writer.writerow(row)
                    count-=1
                elif np.random.randint(100) < 3:
                    writer.writerow(row)
                    count-=1

In [74]:
df_train = pd.read_csv('train.csv')#test.csv')

In [75]:
print('non con: ', sum(df_train.controversiality == 0))
print('con: ', sum(df_train.controversiality == 1))
print ('total: ', len(df_train))

non con:  54423
con:  45577
total:  100000


In [2]:
train = ['/data/reddit1.csv',
         '/data/reddit2.csv',
         '/data/reddit3.csv']

test =  ['/data/reddit9.csv']

finalDf = pd.DataFrame()
finalDf2 = pd.DataFrame()

for file in train:
    df = pd.read_csv(file)
    df = df[['body', 'created_utc', 'controversiality']]
    #adding the controversial 
    controversial = df[df['controversiality'] == 1].copy().reset_index()
    del controversial['index']
    finalDf = finalDf.append(controversial,ignore_index = True)
    #adding the non-controversial 
    non_controversial = df[df['controversiality'] == 0].sample(frac=0.03, replace=False).copy().reset_index()
    del non_controversial['index']
    finalDf = finalDf.append(non_controversial, ignore_index = True)

df_train = finalDf

for file in test:
    df = pd.read_csv(file)
    df = df[['body', 'created_utc', 'controversiality']]
    #adding the controversial 
    controversial = df[df['controversiality'] == 1].copy().reset_index()
    del controversial['index']
    finalDf2 = finalDf2.append(controversial,ignore_index = True)
    #adding the non-controversial 
    non_controversial = df[df['controversiality'] == 0].copy().reset_index()
    del non_controversial['index']
    finalDf2 = finalDf2.append(non_controversial, ignore_index = True)

df_test = finalDf2

del df

df_train = df_train.drop('created_utc', axis=1)
df_train['body'] = df_train['body'].astype(str)

df_test = df_test.drop('created_utc', axis=1)
df_test['body'] = df_test['body'].astype(str)

In [3]:
y_train = df_train.controversiality
y_test = df_test.controversiality

## Functions to extract features

In [4]:
def extractFeature(vectorizer, selector, x_train, y_train, test):
    Vect = vectorizer
    Sel = selector
    training = Vect.fit_transform(x_train)
    test = Vect.transform(test)
    x_train = Sel.fit_transform(training, y_train)
    x_test = Sel.transform(test)
    return x_train, x_test, Vect, Sel

In [6]:
x_train, x_test, vect, sel = extractFeature(TfidfVectorizer(sublinear_tf=True, max_df=0.7, 
                                                 stop_words='english', ngram_range=(0,2)),
                                 SelectKBest(chi2, k=50000),
                                 df_train.body,
                                 df_train.controversiality,
                                 df_test.body)

In [7]:
print(x_train.shape)
print(df_train.shape)
print(x_test.shape)

(158508, 50000)
(158508, 2)
(1000000, 50000)


## Performance boosting time!

In [136]:
intercept = model.intercept_
coefs = model.coef_[0]

In [221]:
%%timeit
    test = vect.transform([row])
    test_select = sel.transform(test)

1 loop, best of 3: 276 ms per loop


In [222]:
%%timeit
    test_select.dot(coefs)

The slowest run took 10.73 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 10.9 µs per loop


In [318]:
test = vect.transform([df_test.body[0]])
print(test)

  (0, 1593646)	0.168452161796
  (0, 1592773)	0.103445173079
  (0, 1583669)	0.0916475154945
  (0, 1542480)	0.18414335362
  (0, 1539685)	0.0823188827085
  (0, 1507017)	0.230735495011
  (0, 1506974)	0.195920834222
  (0, 1505891)	0.211799510322
  (0, 1504451)	0.195920834222
  (0, 1503260)	0.091352313345
  (0, 1401795)	0.238613857281
  (0, 1371347)	0.154853167381
  (0, 1294815)	0.117783769614
  (0, 1250995)	0.203799196493
  (0, 1098375)	0.139403130211
  (0, 1041150)	0.128546383787
  (0, 1039925)	0.0671722860906
  (0, 1012959)	0.22080993239
  (0, 1010434)	0.156063758759
  (0, 819296)	0.139169025787
  (0, 784188)	0.16793398623
  (0, 783903)	0.0762264204764
  (0, 764015)	0.202243932093
  (0, 762153)	0.0599032786312
  (0, 735658)	0.200803982168
  (0, 622326)	0.081929766963
  (0, 518154)	0.203799196493
  (0, 510552)	0.209388983481
  (0, 510511)	0.102304256118
  (0, 459764)	0.173185844142
  (0, 420195)	0.105908013495
  (0, 418899)	0.0658706761646
  (0, 387227)	0.225145708022
  (0, 387031)	0.14837

In [260]:
test_select = sel.transform(test)
print(test_select)

  (0, 4098)	0.0875981887669
  (0, 4781)	0.174574322693
  (0, 6021)	0.138259833867
  (0, 11302)	0.0658706761646
  (0, 18004)	0.081929766963
  (0, 22494)	0.0599032786312
  (0, 24525)	0.139169025787
  (0, 30880)	0.156063758759
  (0, 31625)	0.0671722860906
  (0, 31726)	0.128546383787
  (0, 33614)	0.139403130211
  (0, 39436)	0.117783769614
  (0, 41518)	0.154853167381
  (0, 45721)	0.091352313345
  (0, 45797)	0.211799510322
  (0, 46752)	0.0823188827085
  (0, 48305)	0.0916475154945
  (0, 48569)	0.103445173079


In [264]:
mapping ={}
extractedFeatures = sel.get_support(indices=True)
for i in range(50000):
    mapping[extractedFeatures[i]] = i

In [330]:
%%timeit
test = vect.transform([df_test.body[0]])
pred=0
pred+=intercept
for val, idx in tuple(zip(test.todense()[0, test.nonzero()[1]].tolist()[0], test.nonzero()[1])):
    real_idx = mapping.get(idx)
    if real_idx != None:
        pred+=coefs[real_idx] * val

predict = (1 / (1 + np.exp(-pred)))

100 loops, best of 3: 5.69 ms per loop


## Running Logistic Regression Model

In [15]:
model = LogisticRegressionCV(cv = 2, max_iter=1000, n_jobs=-1)
model.fit(x_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=2, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=1000,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [43]:
print("model fitting score: ",  model.score(x_train, y_train))
print("proportion of controversial posts in training set: ", (sum(y_train) / len(y_train)))
print("validation score: " , model.score(x_test, y_test))
print("proportion of controversial posts in testing set: ", (sum(y_test) / len(y_test)))

model fitting score:  0.82838090191
proportion of controversial posts in training set:  0.445573724985
validation score:  0.775333
proportion of controversial posts in testing set:  0.022174


### Saving our models

In [287]:
from sklearn.externals import joblib
joblib.dump(model, 'tfidftop50K.pkl', compress=9)

['tfidftop50K.pkl']

In [288]:
joblib.dump(vect, 'tfidfmixbigram.pkl', compress=9)

['tfidfmixbigram.pkl']

In [289]:
joblib.dump(sel, 'tfidfselector.pkl', compress=9)

['tfidfselector.pkl']

In [290]:
joblib.dump(mapping, 'tfidfselectormapping.pkl', compress=9)

['tfidfselectormapping.pkl']